# Step 2: Exploratory Data Analysis and Data Set Cleaning

## Import Libraries and Data Set

In [1]:
import pandas as pd
from IPython.core.display import HTML
import numpy as np
import scjpnlib.utils as scjpnutils

In [2]:
# import the data set
kchd_df = pd.read_csv('kc_house_data.csv')

target = 'price'

One of the single most important facets of understanding the data is inspecting its values or, more specifically, the ranges of values each feature has.  Most important in that is understand whether a given feature is quantitative or qualitative, or, more precisely whether a given feature is *continuous* or *categorical*.

But, having null values in the data set may present problems in this part of our understanding process.  So, our first step is to deal with null values.  Even non-null but unexpected anomalous data can present problems and these are known as "outliers".  We must identify when possible and deal with outliers as part of this process as well.

This process is collectively known as "cleaning the data set".
<br>
<br>
Let's have an initial high-level look at what our data looks like.

In [3]:
kchd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(11), object(2)
memory usage: 3.5+ MB


In [4]:
scjpnutils.print_df_head(kchd_df)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


We notice right off the bat that there is a null (`NaN`) value in the very first row.  

But, before we move on to clean those null values, let's get a better understanding of which features are *continuous* (quantitative) vs. which features are *categorical* (qualitative).  This distinction will determine the type of processing that we do on each class, in preparation for building a linear regression model.

<br><br>
## Identify Categorical vs. Continuous Features

Taken directly from the list of feature-definitions, we have:
* **id** - unique identified for a house
* **date** - house was sold
* **price** -  is prediction target
* **bedrooms** -  of Bedrooms/House
* **bathrooms** -  of bathrooms/bedrooms
* **sqft_living** -  footage of the home
* **sqft_lot** -  footage of the lot
* **floors** -  floors (levels) in house
* **waterfront** - House which has a view to a waterfront
* **view** - Has been viewed
* **condition** - How good the condition is ( Overall )
* **grade** - overall grade given to the housing unit, based on King County grading system
* **sqft_above** - square footage of house apart from basement
* **sqft_basement** - square footage of the basement
* **yr_built** - Built Year
* **yr_renovated** - Year when house was renovated
* **zipcode** - zip
* **lat** - Latitude coordinate
* **long** - Longitude coordinate
* **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
* **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors

<br><br>
*Based on the above definitions*, it initially looks like the following features are *categorical* (qualitative):
* **view**
* **waterfront**
* **zipcode**

The rest of the features appear to be *continuous*, *holding quantitive (measured) values*:
* **bedrooms**
* **bathrooms**
* **sqft_living**
* **sqft_lot**
* **floors**
* **condition**
* **grade**
* **sqft_above**
* **sqft_basement**
* **yr_built**
* **yr_renovated**
* **lat**
* **long**
* **sqft_living15**
* **sqft_lot15**

In [5]:
categorical_features = ['waterfront', 'view', 'zipcode']
continuous_features = scjpnutils.get_remaining_features(kchd_df, target, categorical_features)

Now that *categorical* vs. *continuous* have been initially identified, we can move on to actually indentifying those observations that contain nulls values.

Note that, for features containing null values, knowing whether the feature is *continuous* vs. *categorical* has some bearing on deciding how to handle null values.  **We must decide on a replacement strategy if those observations are to be kept**, as retaining null values in most cases will be problematic in building linear regression models.  

Since a "Golden Rule" in Data Science is to avoid dropping observations from a data set unless absolutely necessary, we will opt out of that as a recourse to handling observations with null values.  Thus, we require a *replacement* scheme and knowing whether a feature is continuous or categorical, as well as its set of unique values, will provides some insight.

Having said that, it's time to deal with any null values in the data set.

<br><br>
## Cleaning Null Values

The `cols_with_null_vals` function provides a summary of all features in our data set that have null values.

In [6]:
kchd_cwnv_df = scjpnutils.cols_with_nulls(kchd_df)
display(HTML("<b>Features with null values:</b><br><br>"))
scjpnutils.print_df(kchd_cwnv_df)

,name,index,dtype,n_null
0,waterfront,8,float64,2376
1,view,9,float64,63
2,yr_renovated,15,float64,3842


###  Using Unique Values (and Categorical "Classification") in Null-Value Replacement Guidance

**For null-value replacement, we must treat *continuous* features differently than we do *categorical* features**.

Since *we do not wish to drop any observations containing null values* and since *we do not wish to radically alter their distributions*, **we shall use some aggregate function which retains the existing central tendency of non-null values, for CONTINUOUS features** - i.e. depending on the presence of outliers or not, in the case of *continuous* data, we opt to replace with the *median* or *mean* of the good values, respectively.

**We shall replace null values of CATEGORICAL features with some discrete value from the set of unique non-null *categorical* values**.  But which discrete value?  We must inspect the *meaning* of the categorical values in order to properly choose which non-null value should replace a null value in that case.

The `classify_as_categorical` utility function was written to not only list all unique values of each feature, but it also provides a metric, **p_cat**, related to the ratio of the number of unique values (of a given feature) out of the total number of observations.  In this way, subtracting that ratio from 1 provides a derived "probability" on whether a given feature is *categorical* or not.

Note that, by default, `classify_as_categorical` excludes nulls from the output.

Let's use it now to help determine our replacement heuristic for the features that contain null-valued observations: **waterfront**, **view**, and **yr_renovated**.

In [7]:
kchd__nulls_with_cat_classification_df = scjpnutils.classify_as_categorical(kchd_df[kchd_cwnv_df['name']], p_cat_th=0.0)
display(HTML("<b>Unique values for features containing observations with nulls:</b><br><br>"))
scjpnutils.print_df(kchd__nulls_with_cat_classification_df)

,name,index,dtype,n_unique,p_cat,unique_vals
0,waterfront,0,float64,2,0.9999,"[0.0, 1.0]"
1,view,1,float64,5,0.9998,"[0.0, 1.0, 2.0, 3.0, 4.0]"
2,yr_renovated,2,float64,70,0.9968,"[0.0, 1934.0, 1940.0, 1944.0, 1945.0, 1946.0, 1948.0, 1950.0, 1951.0, 1953.0, 1954.0, 1955.0, 1956.0, 1957.0, 1958.0, 1959.0, 1960.0, 1962.0, 1963.0, 1964.0, 1965.0, 1967.0, 1968.0, 1969.0, 1970.0, 1971.0, 1972.0, 1973.0, 1974.0, 1975.0, 1976.0, 1977.0, 1978.0, 1979.0, 1980.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0, 1986.0, 1987.0, 1988.0, 1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0, 1995.0, 1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0]"


We can see that:
- **waterfront**: since, by definition, it behaves as a *Boolean* value, null values can safely be replaced by 0.0
- **view**: ranges from 0 to 4 but, by definition, it behaves as a *Boolean* value, so in the phase where we are strictly handling null replacement, null values can safely be replaced by 0.0; note that the occurrences of apparently invalid values (other than 0.0 or 1.0) will be handles in a different phase; for now we are handling null-value replacement
- **yr_renovated** clearly uses 0.0 to indicate that a home has not been renovated; so we will adopt the heuristic to replace null with 0.0 in this case

Since we have decided on the heuristics for all features with null values, we can clean those features now.

Note that, on a preliminary basis, it appears **there are no continuous features with null values**.

The `clean_offending_values` utility function was written to facilitate cleaning the data based on a *text-based rule definition*.  It simply uses pandas to locate values that match a given condition - e.g. NaN values - and replaces them with the desired target value.  It is a convenience method for replacing with both literal values and "macro"-based (or computed) values - .e.g. the *mean* or *median* of all non-null values.

Replacement rules are defined according to the *Backus–Naur Form* given below:
- *numeric* replacement rules should be of the form:
    - `{<name_of_col>: \[(outlier_val_1, 'median'|'mean'|<numeric_replacement_value>), ((outlier_val_2, 'median'|'mean'|<numeric_replacement_value>)), ... , (outlier_val_n, 'median'|'mean'|<numeric_replacement_value>)\]}}`
    
*Note that string-value replacement is done similarly, except that there is no notion of aggregate function replacement.*
    
Let's use it now to replace NaN values of the above features with their corresponding values.

In [8]:
# now use the above function to clean 'waterfront' and 'yr_renovated' using lteral-value replacement
scjpnutils.clean_offending_values(
    kchd_df
    , numeric_replacement_rules=
        {
            'waterfront': [(np.NaN, 0.0)]
            , 'view': [(np.NaN, 0.0)]
            , 'yr_renovated': [(np.NaN, 0.0)]
        }
    , friendly_name_of_df="King County House Sales DF"
)

*** CLEANING VALUES of King County House Sales DF: BEGIN ***
Rules for 'waterfront' column value replacement are: [(nan, 0.0)]
Looking for rows with 'waterfront' values [nan] ...
Rows with offending values occur at Int64Index([    0,    10,    23,    40,    55,    60,    62,    66,    87,
              105,
            ...
            21544, 21545, 21550, 21562, 21567, 21578, 21582, 21586, 21587,
            21595],
           dtype='int64', length=2376).
Replaced 2376 offending instances in column 'waterfront' with literal value 0.0

Rules for 'view' column value replacement are: [(nan, 0.0)]
Looking for rows with 'view' values [nan] ...
Rows with offending values occur at Int64Index([    7,   114,   129,   205,   487,   590,   938,  1313,  1364,
             1701,  2415,  2833,  3150,  3237,  4533,  4740,  5196,  5207,
             5393,  5494,  5585,  5746,  6391,  7103,  7269,  7563,  7686,
             7716,  8047,  8066,  8279,  8575,  9405,  9853, 10023, 10174,
            10339

<br><br>
## Handling potential outlier values

Now that we've cleaned all null-values (replaced with non-null values that make sense), we can get some idea of whether a given feature has any outlier values by inspecting its list of unique values.  

We can leverage the `classify_as_categorical` utility function to help identify such outlier values as well as their value ranges.

If we spot any peculiar values we can then deal with them as necessary.

In [9]:
# Let's look for outliers
kchd_cat_classification_df = scjpnutils.classify_as_categorical(kchd_df, p_cat_th=0.0, exclude_null_vals=False)
display(HTML("<b>View of features from the context of their unique values:</b><br><br>"))
scjpnutils.print_df(kchd_cat_classification_df[['name', 'dtype', 'n_unique', 'p_cat', 'unique_vals']].set_index('name'))

,dtype,n_unique,p_cat,unique_vals
name,,,,
id,int64,21420,0.0082,"[1000102, 1200019, 1200021, 2800031, 3600057, 3600072, 3800008, 5200087, 6200017, 7200080, 7200179, 7400062, 7600057, 7600065, 7600125, 7600136, 9000025, 11200070, 11200290, 11200400, 11300120, 11500240, 11500890, 11501160, 11501310, 11501330, 11510310, 11510700, 11520030, 11520200, 11520370, 11520640, 11900140, 13001215, 13001795, 13001991, 13002460, 13002495, 16000015, 16000200, 16000397, 16000435, 16000545, 23500180, 23500190, 23500220, 23520190, 23520380, 31000165, 31200020, 34000005, 34001160, 34001304, 34001540, 34001765, 37000335, 37000435, 40000228, 40000235, 40000362, 40000471, 40000553, 40000669, 40001065, 41000454, 42000006, 42000065, 42000127, 42000130, 42000245, 46100204, 46100350, 46100504, 49000051, 49500090, 50300090, 50300220, 52000067, 53500020, 53500450, 53500760, 56000095, 59000201, 59000250, 59000445, 59500050, 65000085, 65000210, 65000260, 65000400, 66000070, 66000140, 66000265, 84000105, 84000245, 84000335, 87000006, 87000213, 87000245, 87000283, ...]"
date,object,372,0.9828,"[1/10/2015, 1/12/2015, 1/13/2015, 1/14/2015, 1/15/2015, 1/16/2015, 1/17/2015, 1/19/2015, 1/2/2015, 1/20/2015, 1/21/2015, 1/22/2015, 1/23/2015, 1/24/2015, 1/25/2015, 1/26/2015, 1/27/2015, 1/28/2015, 1/29/2015, 1/30/2015, 1/31/2015, 1/5/2015, 1/6/2015, 1/7/2015, 1/8/2015, 1/9/2015, 10/1/2014, 10/10/2014, 10/11/2014, 10/12/2014, 10/13/2014, 10/14/2014, 10/15/2014, 10/16/2014, 10/17/2014, 10/18/2014, 10/19/2014, 10/2/2014, 10/20/2014, 10/21/2014, 10/22/2014, 10/23/2014, 10/24/2014, 10/25/2014, 10/26/2014, 10/27/2014, 10/28/2014, 10/29/2014, 10/3/2014, 10/30/2014, 10/31/2014, 10/4/2014, 10/5/2014, 10/6/2014, 10/7/2014, 10/8/2014, 10/9/2014, 11/1/2014, 11/10/2014, 11/11/2014, 11/12/2014, 11/13/2014, 11/14/2014, 11/15/2014, 11/16/2014, 11/17/2014, 11/18/2014, 11/19/2014, 11/2/2014, 11/20/2014, 11/21/2014, 11/22/2014, 11/23/2014, 11/24/2014, 11/25/2014, 11/26/2014, 11/28/2014, 11/29/2014, 11/3/2014, 11/30/2014, 11/4/2014, 11/5/2014, 11/6/2014, 11/7/2014, 11/8/2014, 11/9/2014, 12/1/2014, 12/10/2014, 12/11/2014, 12/12/2014, 12/13/2014, 12/14/2014, 12/15/2014, 12/16/2014, 12/17/2014, 12/18/2014, 12/19/2014, 12/2/2014, 12/20/2014, 12/21/2014, ...]"
price,float64,3622,0.8323,"[78000.0, 80000.0, 81000.0, 82000.0, 82500.0, 83000.0, 84000.0, 85000.0, 86500.0, 89000.0, 89950.0, 90000.0, 92000.0, 95000.0, 96500.0, 99000.0, 100000.0, 102500.0, 104950.0, 105000.0, 105500.0, 106000.0, 107000.0, 109000.0, 109500.0, 110000.0, 110700.0, 111300.0, 112000.0, 114000.0, 114975.0, 115000.0, 118000.0, 118125.0, 119500.0, 119900.0, 120000.0, 120750.0, 121800.0, 122000.0, 123000.0, 123300.0, 124000.0, 124500.0, 124740.0, 125000.0, 126000.0, 126500.0, 128000.0, 128750.0, 129000.0, 129888.0, 130000.0, 132500.0, 132825.0, 133000.0, 133400.0, 134000.0, 135000.0, 135900.0, 136500.0, 137000.0, 137124.0, 137900.0, 139000.0, 139500.0, 140000.0, 141800.0, 142500.0, 143000.0, 144000.0, 144975.0, 145000.0, 145600.0, 146000.0, 146300.0, 147000.0, 147200.0, 147400.0, 147500.0, 148000.0, 148226.0, 148900.0, 149000.0, 149500.0, 149900.0, 150000.0, 150550.0, 151000.0, 151100.0, 151600.0, 152000.0, 152275.0, 152500.0, 152900.0, 153000.0, 153500.0, 153503.0, 154000.0, 154200.0, ...]"
bedrooms,int64,12,0.9994,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 33]"
bathrooms,float64,29,0.9987,"[0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.0, 5.25, 5.5, 5.75, 6.0, 6.25, 6.5, 6.75, 7.5, 7.75, 8.0]"
sqft_living,int64,1034,0.9521,"[370, 380, 390, 410, 420, 430, 440, 460, 470, 480, 490, 500, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 809, 810, 820, 828, 830, 833, 840, 850, 860, 870, 880, 890, 893, 894, 900, 901, 902, 910, 920, 930, 940, 950, 960, 962, 970, 980, 982, 988, 990, 998, 1000, 1008, 1010, 1020, 1030, 1040, 1048, 1050, 1060, 1061, 1068, 1070, 1072, 1076, 1078, 1080, 1084, 1

### Analysis of Unique Values

Based on the above, we do spot a few specific issues:

1. based on *unique_vals* and *dtype* of **sqft_basement**, it looks like it should be a *numeric* data type but it is of type `Object` (probably stored as text); therefore, **it must be converted to type `float64`**.
2. based on *unique_vals* of ***bedroom*s**, it **has an *apparent* nonsensical OUTLIER value (33) that should probably be replaced with a value that makes sense**.
3. *according to the definition* provided, ***view* is a *boolean* type**, indicating whether a home has been viewed or not, but **it has values other than 0 or 1 which must be mapped to 0 or 1**.
4. the *dtype* of **yr_built** and **yr_renovated** is `int64` and `float64`, respectively; since these features store the same kind of values (year), ***yr_renovated* must be converted to type `int64`**.

Let's address **sqft_basement** first.

### Converting **sqft_basement** from `Object` (string) type to `float64`

Before attempting to convert the data type of **sqft_basement** to `float64`, **we need to replace any potential non-numeric values first**.

Let's find those first.

In [10]:
unique_sqft_basement = kchd_cat_classification_df[kchd_cat_classification_df['name']=='sqft_basement']['unique_vals'].values[0]
non_numeric_sqft_basement_vals = []
for idx, sqft_basement in enumerate(unique_sqft_basement):
    s_val = sqft_basement.strip()
    val = None
    try:       
        val = float(s_val)
    except:
        non_numeric_sqft_basement_vals.append(sqft_basement)

display(HTML("<h3><b>sqft_basement</b> contains the following non-numeric values that must be replaced: {}</h3>".format(non_numeric_sqft_basement_vals)))

<br><br>
We opt to **replace outlier value '?' of the *sqft_basement* feature with '0.0'** since unknown values can be treated as 0.0 numeric.

In [11]:
scjpnutils.clean_offending_values(
    kchd_df
    , string_replacement_rules=
        {
            'sqft_basement': [('?', '0.0')]
        }
    , friendly_name_of_df="King County House Sales DF"
)

*** CLEANING VALUES of King County House Sales DF: BEGIN ***
Rules for 'sqft_basement' are: [('?', '0.0')]
Looking for rows with 'sqft_basement' values in ['?'] ...
Rows with offending values occur at Int64Index([    6,    18,    42,    79,   112,   115,   217,   309,   384,
              508,
            ...
            21236, 21248, 21356, 21357, 21365, 21442, 21447, 21473, 21519,
            21581],
           dtype='int64', length=454).
Replaced 454 offending instances in column 'sqft_basement' with literal value '0.0'
*** CLEANING VALUES of King County House Sales DF: END ***


<br><br>
We can now convert **sqft_basement** to type `float`:

In [12]:
kchd_df['sqft_basement']=kchd_df['sqft_basement'].astype('float64')

<br><br>
### Handling *bedrooms* outliers

Using `kchd_cat_classification_df` to view **bedrooms** unique values, we have the following:

In [13]:
kchd_cat_classification_df[kchd_cat_classification_df['name']=='bedrooms']

,name,index,dtype,n_unique,p_cat,unique_vals
3,bedrooms,3,int64,12,0.9994,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 33]"


<br><br>
Here, we see that there are home(s) that supposedly have 33 bedrooms.  Really?  I don't think so but let's try to justify this mathematically.

Let's examine ALL observations in our data set with **bedrooms** $== 33$.

In [14]:
outlier_rooms = 33
outlier_br_mask = (kchd_df.bedrooms == outlier_rooms)
kchd_with_outlier_rooms_df = kchd_df[outlier_br_mask]
scjpnutils.print_df(kchd_with_outlier_rooms_df)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
15856,2402100895,6/25/2014,640000.0,33,1.75,1620,6000,1.0,0.0,0.0,5,7,1040,580.0,1947,0.0,98103,47.6878,-122.331,1330,4700


<br><br>
First of all, we see that there is only a single observsation with **bedrooms** $== 33$.  This is truly the definition of an outlier. Is it worth it to take the time necessary to clean a single outlier?  Probably not but we do so for the sake of demonstration anyway.

We see that the value of **sqft_living** is only 1620.

Let's compute the average square footage per room and compare its value to the average square footage per room of the entire data set, excluding this possible outlier observation, with **bedrooms**==33.

In [15]:
outlier_rms_avg_sqft_per_room = kchd_with_outlier_rooms_df['sqft_living']/outlier_rooms
outlier_rms_avg_sqft_per_room[outlier_rms_avg_sqft_per_room.index]

15856    49.090909
Name: sqft_living, dtype: float64

<br><br>
We see that, for this outlier, the average square footage per room is about 49 ft.$^2$.  That means, on average, each room is 7 ft. x 7 ft in this property with 33 bedrooms?!?!  Is this likely?  I doubt it but let's compare it to the average square footage per room of the rest of the data set, excluding this possible outlier observation with **bedrooms**==33.

Average square footage per room of the rest of the data set is:

In [16]:
avg_sqft_per_room_excluding_apparent_outliers = kchd_df['sqft_living'].sum()/kchd_df[kchd_df['bedrooms'].isin([outlier_rooms])==False]['bedrooms'].sum()
avg_sqft_per_room_excluding_apparent_outliers

617.0000686643413

<br><br>
We see that our outlier average square footage per room differs *drastically* from the average for the data set.  So we will impute the average of the data set (excluding this outlier observation) to arrive at a reasonable value replacement value for **bedrooms** of the outlier observation.  

Note: So far this has been a fair amount of work done to clean only a single observation but, again, this process demonstrates the potential importance in the general case.

The formula for this is 
$$imputed\_outlier\_bedrooms = \frac{outlier\_sqft\_living}{avg\_sqft\_per\_room\_excluding\_apparent\_outliers}$$ 

... and the value that we will impute in this case is computed as follows:

In [17]:
imputed_outlier_bedrooms_df = kchd_with_outlier_rooms_df['sqft_living']/avg_sqft_per_room_excluding_apparent_outliers
imputed_outlier_bedrooms_df.values[0]

2.62560748738151

<br><br>
Not so fast!  You'll notice that this is a real-valued result.  But does it make sense to have fractional bedrooms?  No it doesn't, not really.

Our work with this outlier is not yet done.

We need to match this imputed real-valued value with its "closest neighbor" in the range of observed *unique* (integer) values of **bedrooms**.  

We will call this set the "permissible values of bedrooms".

The following function accomplishes this:

In [18]:
def permissible_br(br_real):
    unique_bedrooms = sorted(list(kchd_df['bedrooms'].unique()))
    unique_bedrooms.remove(outlier_rooms)
    unique_bedrooms = np.array(unique_bedrooms)
    print("Permissible set of values for 'bedrooms' is: {}".format(unique_bedrooms))
    
    idx_closest_br = -1
    min_dist = -1
    for idx, br in enumerate(unique_bedrooms):
        d = abs(br_real - br)
        if min_dist == -1:
            min_dist = d
            idx_closest_br = idx
        elif d < min_dist:
            min_dist = d
            idx_closest_br = idx
            
    print("Imputing closest permissible bedroom value to computed real-value {}: {}.".format(br_real, unique_bedrooms[idx_closest_br], idx_closest_br))
    return unique_bedrooms[idx_closest_br]

<br><br>
Now we finally impute the corresponding correct value (according to the formula and function above) for all observations with the **bedrooms** outlier value, 33.

In [19]:
kchd_df.loc[outlier_br_mask, 'bedrooms'] = kchd_df.loc[outlier_br_mask].apply(lambda df: permissible_br(df['sqft_living']/avg_sqft_per_room_excluding_apparent_outliers), axis=1)
kchd_df[outlier_br_mask]

Permissible set of values for 'bedrooms' is: [ 1  2  3  4  5  6  7  8  9 10 11]
Imputing closest permissible bedroom value to computed real-value 2.62560748738151: 3.


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
15856,2402100895,6/25/2014,640000.0,3,1.75,1620,6000,1.0,0.0,0.0,...,7,1040,580.0,1947,0.0,98103,47.6878,-122.331,1330,4700


<br><br>
### Handling *view* outliers


The values of **view** greater than 1 should just simply be replaced with 1.

In [20]:
kchd_cat_classification_df[kchd_cat_classification_df['name']=='view'].set_index('name')

,index,dtype,n_unique,p_cat,unique_vals
name,,,,,
view,9,float64,5,0.9998,"[0.0, 1.0, 2.0, 3.0, 4.0]"


In [21]:
unique_views = kchd_cat_classification_df[kchd_cat_classification_df['name']=='view']['unique_vals'].values[0]
positive_vals_gt_zero_replacement_rule = []
for idx, vw in enumerate(unique_views):
    if vw > 1.0:
        positive_vals_gt_zero_replacement_rule.append((vw, 1.0))

display(HTML("<b>view</b> contains the following values > 1.0 values that must be replaced with 1.0: {}".format(positive_vals_gt_zero_replacement_rule[:][0])))

scjpnutils.clean_offending_values(
    kchd_df
    , numeric_replacement_rules=
        {
            'view': positive_vals_gt_zero_replacement_rule
        }
    , friendly_name_of_df="King County House Sales DF"
)

*** CLEANING VALUES of King County House Sales DF: BEGIN ***
Rules for 'view' column value replacement are: [(2.0, 1.0), (3.0, 1.0), (4.0, 1.0)]
Looking for rows with 'view' values [2.0, 3.0, 4.0] ...
Rows with offending values occur at Int64Index([   15,    21,    49,    58,    60,    99,   113,   120,   126,
              153,
            ...
            21489, 21505, 21513, 21523, 21524, 21538, 21560, 21563, 21577,
            21582],
           dtype='int64', length=1782).
Replaced 1782 offending instances in column 'view' with literal value 1.0

*** CLEANING VALUES of King County House Sales DF: END ***


<br><br>
### Convert feature data types as necessary

We noted previously that the *dtype* of **yr_built** and **yr_renovated** is `int64` and `float64`, respectively.  Since these features store the same kind of values (year), we need to convert **yr_renovated** to `int64`.  This is just a straightforward numerical type conversion.  Note that in converting from `float64` to `int64` any existing fractional components will be lost.  But that is okay since we will not be making use of that data - the fractional part (if it exists) of **yr_renovated** - anyway.

In [22]:
kchd_df['yr_renovated']=kchd_df['yr_renovated'].astype('int64')

<br><br>
And with that, we have our final, cleaned data set.

In [23]:
kchd_cat_classification_df = scjpnutils.classify_as_categorical(kchd_df, p_cat_th=0.0, exclude_null_vals=False)
display(HTML("<b>View of features from the context of their unique values:</b><br><br>"))
scjpnutils.print_df(kchd_cat_classification_df[['name', 'dtype', 'n_unique', 'p_cat', 'unique_vals']].set_index('name'))

,dtype,n_unique,p_cat,unique_vals
name,,,,
id,int64,21420,0.0082,"[1000102, 1200019, 1200021, 2800031, 3600057, 3600072, 3800008, 5200087, 6200017, 7200080, 7200179, 7400062, 7600057, 7600065, 7600125, 7600136, 9000025, 11200070, 11200290, 11200400, 11300120, 11500240, 11500890, 11501160, 11501310, 11501330, 11510310, 11510700, 11520030, 11520200, 11520370, 11520640, 11900140, 13001215, 13001795, 13001991, 13002460, 13002495, 16000015, 16000200, 16000397, 16000435, 16000545, 23500180, 23500190, 23500220, 23520190, 23520380, 31000165, 31200020, 34000005, 34001160, 34001304, 34001540, 34001765, 37000335, 37000435, 40000228, 40000235, 40000362, 40000471, 40000553, 40000669, 40001065, 41000454, 42000006, 42000065, 42000127, 42000130, 42000245, 46100204, 46100350, 46100504, 49000051, 49500090, 50300090, 50300220, 52000067, 53500020, 53500450, 53500760, 56000095, 59000201, 59000250, 59000445, 59500050, 65000085, 65000210, 65000260, 65000400, 66000070, 66000140, 66000265, 84000105, 84000245, 84000335, 87000006, 87000213, 87000245, 87000283, ...]"
date,object,372,0.9828,"[1/10/2015, 1/12/2015, 1/13/2015, 1/14/2015, 1/15/2015, 1/16/2015, 1/17/2015, 1/19/2015, 1/2/2015, 1/20/2015, 1/21/2015, 1/22/2015, 1/23/2015, 1/24/2015, 1/25/2015, 1/26/2015, 1/27/2015, 1/28/2015, 1/29/2015, 1/30/2015, 1/31/2015, 1/5/2015, 1/6/2015, 1/7/2015, 1/8/2015, 1/9/2015, 10/1/2014, 10/10/2014, 10/11/2014, 10/12/2014, 10/13/2014, 10/14/2014, 10/15/2014, 10/16/2014, 10/17/2014, 10/18/2014, 10/19/2014, 10/2/2014, 10/20/2014, 10/21/2014, 10/22/2014, 10/23/2014, 10/24/2014, 10/25/2014, 10/26/2014, 10/27/2014, 10/28/2014, 10/29/2014, 10/3/2014, 10/30/2014, 10/31/2014, 10/4/2014, 10/5/2014, 10/6/2014, 10/7/2014, 10/8/2014, 10/9/2014, 11/1/2014, 11/10/2014, 11/11/2014, 11/12/2014, 11/13/2014, 11/14/2014, 11/15/2014, 11/16/2014, 11/17/2014, 11/18/2014, 11/19/2014, 11/2/2014, 11/20/2014, 11/21/2014, 11/22/2014, 11/23/2014, 11/24/2014, 11/25/2014, 11/26/2014, 11/28/2014, 11/29/2014, 11/3/2014, 11/30/2014, 11/4/2014, 11/5/2014, 11/6/2014, 11/7/2014, 11/8/2014, 11/9/2014, 12/1/2014, 12/10/2014, 12/11/2014, 12/12/2014, 12/13/2014, 12/14/2014, 12/15/2014, 12/16/2014, 12/17/2014, 12/18/2014, 12/19/2014, 12/2/2014, 12/20/2014, 12/21/2014, ...]"
price,float64,3622,0.8323,"[78000.0, 80000.0, 81000.0, 82000.0, 82500.0, 83000.0, 84000.0, 85000.0, 86500.0, 89000.0, 89950.0, 90000.0, 92000.0, 95000.0, 96500.0, 99000.0, 100000.0, 102500.0, 104950.0, 105000.0, 105500.0, 106000.0, 107000.0, 109000.0, 109500.0, 110000.0, 110700.0, 111300.0, 112000.0, 114000.0, 114975.0, 115000.0, 118000.0, 118125.0, 119500.0, 119900.0, 120000.0, 120750.0, 121800.0, 122000.0, 123000.0, 123300.0, 124000.0, 124500.0, 124740.0, 125000.0, 126000.0, 126500.0, 128000.0, 128750.0, 129000.0, 129888.0, 130000.0, 132500.0, 132825.0, 133000.0, 133400.0, 134000.0, 135000.0, 135900.0, 136500.0, 137000.0, 137124.0, 137900.0, 139000.0, 139500.0, 140000.0, 141800.0, 142500.0, 143000.0, 144000.0, 144975.0, 145000.0, 145600.0, 146000.0, 146300.0, 147000.0, 147200.0, 147400.0, 147500.0, 148000.0, 148226.0, 148900.0, 149000.0, 149500.0, 149900.0, 150000.0, 150550.0, 151000.0, 151100.0, 151600.0, 152000.0, 152275.0, 152500.0, 152900.0, 153000.0, 153500.0, 153503.0, 154000.0, 154200.0, ...]"
bedrooms,int64,11,0.9995,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
bathrooms,float64,29,0.9987,"[0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.0, 5.25, 5.5, 5.75, 6.0, 6.25, 6.5, 6.75, 7.5, 7.75, 8.0]"
sqft_living,int64,1034,0.9521,"[370, 380, 390, 410, 420, 430, 440, 460, 470, 480, 490, 500, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 809, 810, 820, 828, 830, 833, 840, 850, 860, 870, 880, 890, 893, 894, 900, 901, 902, 910, 920, 930, 940, 950, 960, 962, 970, 980, 982, 988, 990, 998, 1000, 1008, 1010, 1020, 1030, 1040, 1048, 1050, 1060, 1061, 1068, 1070, 1072, 1076, 1078, 1080, 1084, 1088,

<br><br>
The last thing that remains is to save the cleaned data set to a file.

In [24]:
fname = 'kc_house_cleaned_data.csv'
kchd_df.set_index('id').to_csv('kc_house_cleaned_data.csv', sep=',')
print("updated {}".format(fname))

updated kc_house_cleaned_data.csv
